# Part 1

## Additional experiment

In this version, all auxiliary units are removed.

In [8]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split


class MLPnet(nn.Module):  # Changed here
    def __init__(self, input_size=3 * 32 * 32, num_class=10):
        super(MLPnet, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
        )
        self.classifier = nn.Linear(128, num_class)

    def forward(self, x):
        feature = self.model(x)
        out = self.classifier(feature)
        return out


transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
model = MLPnet() 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

In [10]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_accuracy = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')


def test(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.4f}')


train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)
test(model, test_loader)

Epoch [1/20], Loss: 1.8945, Validation Accuracy: 0.3611
Epoch [2/20], Loss: 1.7336, Validation Accuracy: 0.3903
Epoch [3/20], Loss: 1.6617, Validation Accuracy: 0.4035
Epoch [4/20], Loss: 1.6085, Validation Accuracy: 0.4103
Epoch [5/20], Loss: 1.5746, Validation Accuracy: 0.4309
Epoch [6/20], Loss: 1.5546, Validation Accuracy: 0.4419
Epoch [7/20], Loss: 1.5255, Validation Accuracy: 0.4523
Epoch [8/20], Loss: 1.5007, Validation Accuracy: 0.4535
Epoch [9/20], Loss: 1.4861, Validation Accuracy: 0.4559
Epoch [10/20], Loss: 1.4765, Validation Accuracy: 0.4751
Epoch [11/20], Loss: 1.4624, Validation Accuracy: 0.4685
Epoch [12/20], Loss: 1.4471, Validation Accuracy: 0.4714
Epoch [13/20], Loss: 1.4347, Validation Accuracy: 0.4766
Epoch [14/20], Loss: 1.4284, Validation Accuracy: 0.4759
Epoch [15/20], Loss: 1.4177, Validation Accuracy: 0.4862
Epoch [16/20], Loss: 1.4118, Validation Accuracy: 0.4913
Epoch [17/20], Loss: 1.3996, Validation Accuracy: 0.4862
Epoch [18/20], Loss: 1.3906, Validation 

In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model = MLPnet() 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)
test(model, test_loader)

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/20], Loss: 2.0129, Validation Accuracy: 0.2807
Epoch [2/20], Loss: 1.8478, Validation Accuracy: 0.3529
Epoch [3/20], Loss: 1.7588, Validation Accuracy: 0.3794
Epoch [4/20], Loss: 1.6737, Validation Accuracy: 0.3864
Epoch [5/20], Loss: 1.6180, Validation Accuracy: 0.4179
Epoch [6/20], Loss: 1.5715, Validation Accuracy: 0.4452
Epoch [7/20], Loss: 1.5302, Validation Accuracy: 0.4377
Epoch [8/20], Loss: 1.4982, Validation Accuracy: 0.4382
Epoch [9/20], Loss: 1.4681, Validation Accuracy: 0.4458
Epoch [10/20], Loss: 1.4446, Validation Accuracy: 0.4582
Epoch [11/20], Loss: 1.4090, Validation Accuracy: 0.4786
Epoch [12/20], Loss: 1.3938, Validation Accuracy: 0.4844
Epoch [13/20], Loss: 1.3784, Validation Accuracy: 0.4748
Epoch [14/20], Loss: 1.3455, Validation Accuracy: 0.4697
Epoch [15/20], Loss: 1.3297, Validation Accuracy: 0.4814
Epoch [16/20], Loss: 1.3069, Validation Accuracy: 0.4781
Epoch [17/20], Loss: 

In [12]:
from MLPnet import MLPnet as Mlpnet
model = Mlpnet() 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

train(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)
test(model, test_loader)

Epoch [1/20], Loss: 1.7449, Validation Accuracy: 0.4255
Epoch [2/20], Loss: 1.5250, Validation Accuracy: 0.4490
Epoch [3/20], Loss: 1.4273, Validation Accuracy: 0.4511
Epoch [4/20], Loss: 1.3558, Validation Accuracy: 0.4306
Epoch [5/20], Loss: 1.3004, Validation Accuracy: 0.4420
Epoch [6/20], Loss: 1.2495, Validation Accuracy: 0.5079
Epoch [7/20], Loss: 1.2026, Validation Accuracy: 0.4666
Epoch [8/20], Loss: 1.1640, Validation Accuracy: 0.4627
Epoch [9/20], Loss: 1.1201, Validation Accuracy: 0.4944
Epoch [10/20], Loss: 1.0886, Validation Accuracy: 0.5023
Epoch [11/20], Loss: 1.0444, Validation Accuracy: 0.5172
Epoch [12/20], Loss: 1.0082, Validation Accuracy: 0.5053
Epoch [13/20], Loss: 0.9673, Validation Accuracy: 0.5283
Epoch [14/20], Loss: 0.9263, Validation Accuracy: 0.4993
Epoch [15/20], Loss: 0.8920, Validation Accuracy: 0.4749
Epoch [16/20], Loss: 0.8557, Validation Accuracy: 0.5029
Epoch [17/20], Loss: 0.8245, Validation Accuracy: 0.5114
Epoch [18/20], Loss: 0.7918, Validation 